In [1]:
import cobra
from cobra.medium import minimal_medium
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import numpy as np
import pandas as pd

In [2]:
'''load the base model for REL606.'''
base_model = cobra.io.read_sbml_model('../metabolic-modeling/iECB_1328.xml.gz')

max_growth = base_model.slim_optimize()
base_model.medium = minimal_medium(base_model, max_growth)

In [3]:
glucose_model = base_model.copy()
citrate_model = base_model.copy()
## set glucose to zero, and citrate to 10.0.
DM0 = citrate_model.medium
DM0['EX_glc__D_e'] = 0.0
DM0['EX_cit_e'] = 10.0
citrate_model.medium = DM0

assert 'EX_cit_e' in citrate_model.medium
assert 'EX_glc__D_e' not in citrate_model.medium
assert 'EX_cit_e' not in glucose_model.medium
assert 'EX_glc__D_e' in glucose_model.medium

In [4]:
''' consider comparing ATPM objective against biomass objective. '''
##glucose_model.objective = "ATPM"
##citrate_model.objective = "ATPM"

glucose_fba_solution = glucose_model.optimize()
citrate_fba_solution = citrate_model.optimize()

In [5]:
citrate_fba_solution

,fluxes,reduced_costs
DM_4crsol_c,1.074086e-04,0.000000e+00
DM_5drib_c,1.083720e-04,0.000000e+00
DM_aacald_c,0.000000e+00,0.000000e+00
DM_amob_c,9.633062e-07,0.000000e+00
DM_mththf_c,2.157806e-04,0.000000e+00
...,...,...
URFGTT,0.000000e+00,-2.809823e-01
3HBCOAHL,0.000000e+00,0.000000e+00
DMALRED,1.052841e+00,4.857226e-17
AMMQT8_2,0.000000e+00,0.000000e+00


In [6]:
citrate_model.summary()

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
o2_e    11.4       for_e  25.6   BIOMASS_Ec_i...  0.482
cit_e   10         co2_e  14.7
nh4_e    5.2       h2o_e   7.37
pi_e     0.465
so4_e    0.121
k_e      0.094
fe2_e    0.00774
mg2_e    0.00418
cl_e     0.00251
ca2_e    0.00251
cu2_e    0.000341
mn2_e    0.000333
zn2_e    0.000164
ni2_e    0.000156
mobd_e   6.21e-05


In [7]:
DM0_pfba_solution = cobra.flux_analysis.pfba(citrate_model)
DM25_pfba_solution = cobra.flux_analysis.pfba(glucose_model)

In [8]:
''' add this reaction to maeA: nad[c] + oaa[c] ->pyr[c] + nadh[c].
    This model is the difference between citrate_model2 and citrate_model.'''


maeA_side_reaction = cobra.Reaction('MAEA')
maeA_side_reaction.name = 'maeA_side_rxn '
maeA_side_reaction.subsystem = 'Anaplerotic Reactions'
maeA_side_reaction.lower_bound = 0.  # This is the default
maeA_side_reaction.upper_bound = 1000.  # This is the default

citrate_model2 = citrate_model.copy()
citrate_model2.add_reaction(maeA_side_reaction)

maeA_side_reaction.add_metabolites({
    'nad_c': -1.0,
    'oaa_c': -1.0,
    'pyr_c': 1.0,
    'nadh_c': 1.0,
})

maeA_side_reaction.reaction



'nad_c + oaa_c --> nadh_c + pyr_c'

In [26]:
DM25_citrate_model = citrate_model2.copy()
DM25 = DM25_citrate_model.medium
DM25['EX_glc__D_e'] = 1.0
DM25['EX_cit_e'] = 10.0
DM25_citrate_model.medium = DM0


In [27]:

citrate_fba_solution2 = citrate_model2.optimize()
DM0_pfba_solution2 = cobra.flux_analysis.pfba(citrate_model2)
DM25_citrate_fba_solution = DM25_citrate_model.optimize()

In [10]:
citrate_model2.summary()

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
o2_e    15.8       for_e  25.1   BIOMASS_Ec_i...  0.536
cit_e   10         h2o_e  20
nh4_e    5.79
pi_e     0.517
so4_e    0.135
k_e      0.105
fe2_e    0.00861
mg2_e    0.00465
cl_e     0.00279
ca2_e    0.00279
cu2_e    0.00038
mn2_e    0.00037
zn2_e    0.000183
ni2_e    0.000173
mobd_e   6.91e-05


In [11]:
citrate_model.metabolites.pyr_c.summary()
citrate_model.metabolites.mal__L_c.summary()
citrate_model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
77%  20.8    POR5      coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_c...
23%   6.19   ME2       mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c

CONSUMING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
95%  25.6    PFL       coa_c + pyr_c --> accoa_c + for_c
3%    0.842  ACLS      h_c + 2.0 pyr_c --> alac__S_c + co2_c
1%    0.28   ALATA_L   akg_c + ala__L_c <=> glu__L_c + pyr_c
PRODUCING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------------------------
100%   13.2   FUM       fum_c + h2o_c <=> mal__L_c

CONSUMING REACTIONS -- L-Malate (mal__L_c)
---------------------

In [12]:
citrate_model2.metabolites.pyr_c.summary()
citrate_model2.metabolites.mal__L_c.summary()
citrate_model2.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
51%  13.7    POR5      coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_c...
49%  12.9    MAEA      nad_c + oaa_c --> nadh_c + pyr_c

CONSUMING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
94%  25.1    PFL       coa_c + pyr_c --> accoa_c + for_c
4%    0.937  ACLS      h_c + 2.0 pyr_c --> alac__S_c + co2_c
2%    0.538  ALATA_L   akg_c + ala__L_c <=> glu__L_c + pyr_c
PRODUCING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
63%   12.5   FUM       fum_c + h2o_c <=> mal__L_c
37%    7.17  MALS      accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c

C

In [13]:
citrate_model2.metabolites.cit_c.summary()
citrate_model2.metabolites.acon_C_c.summary()
citrate_model2.metabolites.icit_c.summary()
citrate_model2.metabolites.accoa_c.summary()

PRODUCING REACTIONS -- Citrate (cit_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
80%    10     CITt7pp   cit_p + succ_c --> cit_c + succ_p
20%     2.48  CS        accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c

CONSUMING REACTIONS -- Citrate (cit_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%   12.5   ACONTa    cit_c <=> acon_C_c + h2o_c
PRODUCING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------
100%    12.5  ACONTa    cit_c <=> acon_C_c + h2o_c

CONSUMING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------
100%    12.5  ACONTb    acon_C_c + h2o

In [32]:
citrate_model2.reactions.CITL

Reaction identifier,CITL
Name,Citrate lyase
Memory address,0x013167ec8d0
Stoichiometry,cit_c --> ac_c + oaa_c Citrate --> Acetate + Oxaloacetate
GPR,ECB_00585 and ECB_00583 and ECB_00584 and ECB_00582
Lower bound,0.0
Upper bound,1000.0


In [34]:
print(citrate_fba_solution2['CITL'])
print(citrate_)

0.0

In [29]:
DM25_citrate_model.metabolites.cit_c.summary()
DM25_citrate_model.metabolites.acon_C_c.summary()
DM25_citrate_model.metabolites.icit_c.summary()
DM25_citrate_model.metabolites.accoa_c.summary()
DM25_citrate_model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- Citrate (cit_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
80%    10     CITt7pp   cit_p + succ_c --> cit_c + succ_p
20%     2.48  CS        accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c

CONSUMING REACTIONS -- Citrate (cit_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%   12.5   ACONTa    cit_c <=> acon_C_c + h2o_c
PRODUCING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------
100%    12.5  ACONTa    cit_c <=> acon_C_c + h2o_c

CONSUMING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------
100%    12.5  ACONTb    acon_C_c + h2o

In [14]:
citrate_model.metabolites.mal__L_c.summary()
citrate_model.metabolites.oaa_c.summary()
citrate_model.metabolites.pyr_c.summary()
citrate_model.metabolites.cit_c.summary()

PRODUCING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------------------------
100%   13.2   FUM       fum_c + h2o_c <=> mal__L_c

CONSUMING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------------------------
47%     6.19  ME2       mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
45%     5.99  MDH       mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
8%      1.05  DMALRED   fad_c + mal__L_c --> fadh2_c + oaa_c
PRODUCING REACTIONS -- Oxaloacetate (oaa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -----------------------------------------------
85%    5.99  MDH       mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
15%    1.05  DMALRED   fad_c + mal__L_c --> fadh2_c + oaa_c

CONSUMING REACTIONS -- Oxaloacetate (oaa_c)
------------

In [15]:
glucose_model.metabolites.mal__L_c.summary()
glucose_model.metabolites.oaa_c.summary()
glucose_model.metabolites.pyr_c.summary()
glucose_model.metabolites.accoa_c.summary()

PRODUCING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
100%    4.82  FUM       fum_c + h2o_c <=> mal__L_c

CONSUMING REACTIONS -- L-Malate (mal__L_c)
------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
55%     2.68  MDH       mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
45%     2.15  DMALRED   fad_c + mal__L_c --> fadh2_c + oaa_c
PRODUCING REACTIONS -- Oxaloacetate (oaa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -----------------------------------------------
38%    2.91  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
35%    2.68  MDH       mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
28%    2.15  DMALRED   fad_c + mal__L_c --> fadh2_c + oaa_c

CONSUMING REACTIONS -- Oxaloacetate (oaa_c)
----------------------

In [16]:
''' look at effects of single and double gene and reaction knockouts.'''
single_gene_KO_results = single_gene_deletion(citrate_model2, return_frame=True)
single_rxn_KO_results = single_reaction_deletion(citrate_model2, return_frame=True)

In [17]:
##double_gene_KO_results = double_gene_deletion(citrate_model2, return_frame=True)

In [18]:
##double_rxn_KO_results = double_reactions_deletion(citrate_model2, return_frame=True)

In [19]:
citrate_fba_solution2

,fluxes,reduced_costs
DM_4crsol_c,0.000119,0.000000e+00
DM_5drib_c,0.000121,0.000000e+00
DM_aacald_c,0.000000,0.000000e+00
DM_amob_c,0.000001,0.000000e+00
DM_mththf_c,0.000240,0.000000e+00
...,...,...
3HBCOAHL,0.000000,0.000000e+00
DMALRED,1.171222,-8.326673e-17
AMMQT8_2,0.000000,-2.938272e-01
PPOR,0.000000,2.775558e-17


In [20]:
glucose_model2 = glucose_model.copy()
glucose_model2.add_reaction(maeA_side_reaction)
glucose_fba_solution2 = glucose_model2.optimize()

In [21]:
glucose_fba_solution2

,fluxes,reduced_costs
DM_4crsol_c,0.000219,0.000000e+00
DM_5drib_c,0.000221,0.000000e+00
DM_aacald_c,0.000000,0.000000e+00
DM_amob_c,0.000002,0.000000e+00
DM_mththf_c,0.000440,0.000000e+00
...,...,...
3HBCOAHL,0.000000,0.000000e+00
DMALRED,2.147590,6.937330e-17
AMMQT8_2,0.000000,-1.110223e-16
PPOR,0.000000,-5.923989e-13


In [22]:
glucose_fba_solution

,fluxes,reduced_costs
DM_4crsol_c,0.000219,0.000000e+00
DM_5drib_c,0.000221,0.000000e+00
DM_aacald_c,0.000000,0.000000e+00
DM_amob_c,0.000002,0.000000e+00
DM_mththf_c,0.000440,0.000000e+00
...,...,...
URFGTT,0.000000,-1.935384e-13
3HBCOAHL,0.000000,1.110223e-16
DMALRED,2.147590,3.739075e-18
AMMQT8_2,0.000000,-1.110223e-16


In [23]:
g_flux = glucose_fba_solution2.fluxes
c_flux = citrate_fba_solution2.fluxes

g_flux_norm = np.sqrt(np.square(g_flux).sum())
c_flux_norm = np.sqrt(np.square(c_flux).sum())

In [24]:
flux_diff = c_flux - g_flux
flux_diff2 = flux_diff[np.abs(flux_diff) > 5]
print(len(flux_diff2))
flux_diff2.sort_values()

40


EX_h2o_e       -25.609232
FORtex         -25.076853
EX_co2_e       -19.670846
GAPD           -17.563637
ATPS4rpp       -17.556249
ENO            -16.796841
POR5           -16.429884
NADH16pp       -11.526452
TARTRt7pp      -10.000000
GLCtex_copy1   -10.000000
GLCptspp       -10.000000
EX_cit_e       -10.000000
EX_h_e          -9.027240
TPI             -8.186825
PDH             -7.967447
FBA             -7.082600
PFK             -6.616169
PGI             -5.916320
ACONTb           7.626828
ACONTa           7.626828
FUM              7.641277
SUCDi            7.781785
Htex             9.027240
CITtex          10.000000
CITt7pp         10.000000
TARTRtpp        10.000000
EX_glc__D_e     10.000000
MAEA            12.929011
MDH             13.132857
THD2pp          15.513836
FLDR2           16.337511
PGM             16.796841
PGK             17.563637
CO2tex          19.670846
CO2tpp          19.670846
PFL             25.075539
FORtppi         25.076853
EX_for_e        25.076853
H2Otpp      

In [25]:
g_flux.dot(c_flux)/(g_flux_norm*c_flux_norm)

c_proj_on_g = (c_flux.dot(g_flux)/c_flux_norm) * c_flux
c_perp = c_flux - c_proj_on_g
c_perp2 = c_perp[c_perp != 0].

SyntaxError: invalid syntax (<ipython-input-25-19eab681b898>, line 5)

In [ ]:
c_perp2.sort_values()

In [ ]:
''' knock out only those reactions with no flow under glucose.'''

c_perp_rxns = list(c_perp2.index)
KO_reactions = [x for x in citrate_model2.reactions if x._id in c_perp_rxns]

single_rxn_perp_KO_results = single_reaction_deletion(citrate_model2, KO_reactions)
singleKO_results2 = single_rxn_perp_KO_results.copy()
singleKO_results2['comp'] = 0.536 - singleKO_results2['growth']
singleKO_results2['comp'].sort_values()

In [ ]:
''' let us look for reactions which increase flux in response to higher glucose input,
    but decrease flux in response to higher citrate input, or vice-versa.
    These would be candidate reactions for antagonistic pleiotropy.'''

## increment glucose or citrate influx from 10.0 to 11.0.
## note that result is somewhat sensitive to +/- epsilon: MAEA is pleiotropic for -e but not +e.
epsilon = 0.5

glucose_model_dx = glucose_model2.copy()
citrate_model_dy = citrate_model2.copy()
DM25_dx = glucose_model2.medium
DM0_dy = citrate_model2.medium
DM25_dx['EX_glc__D_e'] = DM25_dx['EX_glc__D_e'] - epsilon
DM0_dy['EX_cit_e'] = DM0_dy['EX_cit_e'] - epsilon
glucose_model_dx.medium = DM25_dx
citrate_model_dy.medium = DM0_dy
glucose_dx_solution = glucose_model_dx.optimize()
citrate_dy_solution = citrate_model_dy.optimize()

dCdy = citrate_dy_solution.fluxes - citrate_fba_solution2.fluxes
dGdx = glucose_dx_solution.fluxes - glucose_fba_solution2.fluxes

flux_derivative = pd.DataFrame({'dCdy':dCdy,'dGdx':dGdx})
flux_derivative['vec_norm'] = np.sqrt(np.square(flux_derivative).sum(axis=1))
flux_derivative['product'] = flux_derivative['dCdy']*flux_derivative['dGdx']

In [ ]:
flux_derivative2 = flux_derivative[flux_derivative['vec_norm'] > epsilon]

In [ ]:
''' this result shows no tradeoff between glucose and citrate, neglecting the costs of gene expression.
    what if I fix oxaloacetate decarboxylation at some value? Perhaps some tradeoff. But gene regulation could
    alleviate the tradeoff.'''
dGdx_norm = np.sqrt(np.square(flux_derivative['dGdx']).sum())
dCdy_norm = np.sqrt(np.square(flux_derivative['dCdy']).sum())
theta = np.arccos(flux_derivative2['dCdy'].dot(flux_derivative2['dGdx'])/(dGdx_norm*dCdy_norm))
print(theta)
flux_derivative2.sort_values('vec_norm').round(2)
flux_derivative2.sort_values('product').round(2)
flux_derivative2.sort_values('dCdy').round(2)

In [ ]:
''' save FBA model in JSON and write to file. Then play with escher.github.io.'''
cobra.io.save_json_model(citrate_model2, "../metabolic-modeling/citrate2.json")

In [ ]:
''' measure the cost of maeA flux if fixed to the same value as in DM0.'''
glucose_model3 = glucose_model2.copy()
glucose_model3.reactions.MAEA.lower_bound = 10
test = glucose_model3.optimize()
test